<a href="https://colab.research.google.com/github/doppelgangerlynx/digit-recognizer/blob/master/MNIST_do.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Import dependencies#

In [0]:
import tensorflow as tf
import numpy as np
import os
import pprint
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Dropout, Flatten 
import pandas as pd
import matplotlib.pyplot as plt

#2.Test TPUs#

In [0]:
devices=None
try:
    device_name = os.environ['COLAB_TPU_ADDR']
    TPU_addr = 'grpc://' + device_name
    print('TPU URL:', TPU_addr)
    with tf.Session(TPU_addr) as session:
        devices=session.list_devices()
    print('TPU devices:')
    pprint.pprint(devices)
except KeyError:
    print('TPU not installed in this runtime.')
    
  

#3.AUTH for google drive#

In [0]:
#import os
from google.colab import drive
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/My Drive/MNIST_custom")
!ls

#4. Reading from CSVs#

In [0]:
training_set = np.genfromtxt('train.csv',delimiter=',')[1:]
print(training_set.shape)
y_train=training_set[:,0]
x_train\
    =training_set[:, 1:].reshape((training_set.shape[0], 28, 28, 1)).astype('float32')/255.0
print("x_train.shape",x_train.shape)
print("y_train.shape",y_train.shape)


testing_set = np.genfromtxt('test.csv',delimiter=',')[1:].astype('float32')
print(testing_set.shape)
# y_test=testing_set[:,0]
x_test=testing_set[:,:].reshape((testing_set.shape[0],28, 28, 1)).astype('float32')/255.0
print("x_test.shape",x_test.shape)
# print(y_test.shape)

plt.gray()

plt.title(str(y_train[0]) + ' shown')
sample_plot = plt.imshow(x_train[0][:, :, 0])
# print(y_train[0],'shown')

#5. Build the MNIST CNN#

In [0]:
model =tf.keras.models.Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28,1)),
    # Note: the first layer has to be given the input shape. After that is very automatic.
    Conv2D(32, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(.2),
    
    Conv2D(64, (3, 3), activation='relu'),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(.2),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(.25),
    Dense(10, activation='softmax')    
])
model.summary()

#6.Keras model to TPU: conversion#

In [0]:
tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(tpu=TPU_addr)
    )
)
tpu_model.compile(
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-3),
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    metrics=['sparse_categorical_accuracy']
)


# generator: A generator or an instance of Sequence (keras.utils.Sequence) object in order to avoid duplicate data when using multiprocessing. The output of the generator must be either

# a tuple (inputs, targets)
# a tuple (inputs, targets, sample_weights).
def train_gen(batch_size):
    # the reason for using the generator function is because it does not allocate new function stacks, thus efficient
    while True:
        # First, we choose the first random starting index for the random batch
        offset = np.random.randint(0, x_train.shape[0] - batch_size)
        yield x_train[offset:offset+batch_size], y_train[offset:offset+batch_size]


#7. Train the model and time it#

In [0]:
%%time
history = tpu_model.fit_generator(
    train_gen(128),
    epochs=30,
    steps_per_epoch=100
)

In [0]:
type(tpu_model)

In [0]:
print(history.history)

In [0]:
# def plot_hist(train_loss, train_acc, test_loss, test_acc):
#   fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))
  
#   ax1.plot(train_loss, label='train loss')
#   ax1.plot(test_loss, label='test loss')
#   ax1.set_xlabel('Epochs')
#   ax1.set_ylabel('Loss')
#   ax1.set_title('Train & Test Loss')
#   ax1.legend()
  
  
#   ax2.plot(train_acc, label='train acc')
#   ax2.plot(test_acc, label='test acc')
#   ax2.set_xlabel('Epochs')
#   ax2.set_ylabel('Accuracy')
#   ax2.set_title('Train & Test Accuracy')
#   ax2.legend()
    
#   plt.show()



def plot_hist(train_loss):
  fig, (ax1) = plt.subplots(1, 1, figsize=(4, 4))
  
  ax1.plot(train_loss, label='train loss')
  ax1.set_xlabel('Epochs')
  ax1.set_ylabel('Loss')
  ax1.legend()
    
  plt.show()


In [0]:
plot_hist(history.history['loss'])

#8.Predict the test data#

In [0]:
predicted_prob = tpu_model.predict(x_test)
predicted_classes = np.argmax(predicted_prob, axis=1)
print(predicted_classes)

In [0]:
type(predicted_classes)

print(np.random.randint(predicted_classes.shape[0], size=10).tolist())
import time
for i in np.random.randint(predicted_classes.shape[0], size=10).tolist():
    plt.title(str(predicted_classes[i]) + ' shown')
    sample_plot = plt.imshow(x_test[i][:, :, 0])
#     time.sleep(0.01)
    plt.show()



In [0]:
indices = np.array(range(1,predicted_classes.shape[0] + 1)).reshape(predicted_classes.shape[0], 1)
v_pred = predicted_classes.reshape(predicted_classes.shape[0], 1)
print(v_pred)
result_df = pd.DataFrame(np.hstack((indices, v_pred)), columns=['ImageId', 'Label'])
print(result_df)

In [0]:
result_df.to_csv('submission.csv', encoding='utf-8', index=False)

In [0]:
!ls

# EXTRA: data augmentation#

In [0]:
# CREATE MORE IMAGES VIA DATA AUGMENTATION
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rotation_range=10,  
        zoom_range = 0.10,  
        width_shift_range=0.1, 
        height_shift_range=0.1)

In [0]:
#Train again, with the IDG function!
%%time
history = tpu_model.fit_generator(
    datagen.flow(x_train,y_train, batch_size=128),
    epochs=20,
    steps_per_epoch=x_train.shape[0]//128
)